# See which have completed 

In [1]:
# imports 
import pandas as pd 
import pickle
import glob
import re
from collections import Counter
from Bio import SeqIO
import seaborn as sns

In [2]:
# out of the AlphaFold completed proteins see what still needs to be done 
with open('../files/phrogs_pass_plddt_length_threshold.txt', 'r') as file:
    phrogs_filtered =  [line.strip() for line in file]
    
# proteins in the validation set
alphafold_known_state = pickle.load(open('/home/grig0076/GitHubs/phlegm/files/phrogs_pass_plddt_length_threshold_oligomeric_state.pkl', 'rb'))
known_state_keys = list(alphafold_known_state.keys())

In [3]:
# read in the completed paths 
# read in the metrics generated for the 25,000+ phrogs
part_1 = glob.glob('/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/nonsingleton_representative_sequences.part_001.phlegm/metrics_pickle/*.pkl')
part_2 = glob.glob('/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/nonsingleton_representative_sequences.part_002.phlegm/metrics_pickle/*.pkl')
part_3 = glob.glob('/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/nonsingleton_representative_sequences.part_003.phlegm/metrics_pickle/*.pkl')
part_4 = glob.glob('/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/nonsingleton_representative_sequences.part_004.phlegm/metrics_pickle/*.pkl')
part_5 = glob.glob('/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/nonsingleton_representative_sequences.part_005.phlegm/metrics_pickle/*.pkl')
part_6 = glob.glob('/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/nonsingleton_representative_sequences.part_006.phlegm/metrics_pickle/*.pkl') 
part_7 = glob.glob('/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/nonsingleton_representative_sequences.part_007.phlegm/metrics_pickle/*.pkl')
part_8 = glob.glob('/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/nonsingleton_representative_sequences.part_008.phlegm/metrics_pickle/*.pkl')
all_parts =  part_1 + part_2 + part_3 + part_4 + part_5 + part_6 + part_7 + part_8

In [4]:
all_phrogs_pickle_complete = [re.split('/', p)[-1][:-4] for p in all_parts] 

# make a dictionary that map|s the phrog to its metrics path 
pickle_paths = dict(zip(all_phrogs_pickle_complete, all_parts))

In [5]:
# get the proteins in the 876 subset that didn't complete 
subunit_strings = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
required_a3ms = [] 
phrogs_updating = [] 
phrogs_updating_paths = [] 
 
# will need to recompute stuff for the things here that failed 
for k in known_state_keys: 
 
    metrics = pickle.load(open(pickle_paths.get(k), 'rb'))
    subunits_present = [re.split('_', i)[0] for i in list(metrics.keys())]
    subunits_missing = set(subunit_strings) - set(subunits_present) 
    
    for s in list(subunits_missing): 
        string = k + '.' + s + '.a3m'
        if k == 'phrog_32676': 
            print(k)
        required_a3ms.append(string)
        
    if len(subunits_missing) > 0: 
        phrogs_updating.append(k)
        phrogs_updating_paths.append(pickle_paths.get(k)) 
        
# Save the list to a text file
with open('required_a3ms.txt', 'w') as f:
    for item in required_a3ms:
        f.write(f"{item}\n")
        
with open('paths_updating.txt', 'w') as f: 
    for item in phrogs_updating_paths: 
        f.write(f"{item}\n")

In [6]:
# import the sequences so can see how long they are 
fasta_dict = {record.id: str(record.seq) for record in SeqIO.parse("../files/nonsingleton_representative_sequences.fasta", "fasta")}

# get the length of each phrog 
seq_lengths = [len(f) for f in list(fasta_dict.values())]

# fetch the lengths that could not be folded up to a 10-mer complex 
failed_length = [len(fasta_dict.get(p)) for p in phrogs_updating] 

## check the progress of the ones that should be included 

In [9]:
filtered_phrogs = pd.read_csv('../files/phrogs_pass_plddt_length_threshold.txt', header=None)[0].to_list()

subunit_strings = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
replicate_strings = ['rep_1', 'rep_2', 'rep_3', 'rep_4', 'rep_5'] 
required_a3ms = [] 
phrogs_updating = [] 
phrogs_updating_paths = [] 
 
# will need to recompute stuff for the things here that failed 
for k in filtered_phrogs: 
    
 
    if k in list(pickle_paths.keys()): 
        
        metrics = pickle.load(open(pickle_paths.get(k), 'rb'))
        subunits_present = [re.split('_', i)[0] for i in list(metrics.keys())]
        subunits_missing = set(subunit_strings) - set(subunits_present) 

        for s in list(subunits_missing): 
            string = k + '.' + s + '.a3m'
            required_a3ms.append(string)

        if len(subunits_missing) > 0: 
            phrogs_updating.append(k)
            phrogs_updating_paths.append(pickle_paths.get(k))
            
        for i in list(metrics.keys()): 
            replicates_present = list(metrics.get(i).keys())
            replicates_missing = set(replicate_strings) - set(replicates_present)  
            if len(replicates_missing) > 0: 
                phrogs_updating.append(k) 
                s = re.split('_',i)[0]
                string = k + '.' + s + '.a3m'
                required_a3ms.append(string) # add this here 
            
    else: 
     
        for s in list(subunit_strings): 
            string = k + '.' + s + '.a3m'
            required_a3ms.append(string)

In [8]:
# Save 'required_a3ms' to a text file
with open('../files/required_a3ms.txt', 'w') as file:
    for item in required_a3ms:
        file.write(f"{item}\n")

# Save 'phrogs_updating' to a text file
with open('../files/phrogs_updating.txt', 'w') as file:
    for item in phrogs_updating:
        file.write(f"{item}\n")

In [95]:
set([re.split('\.',p )[1] for p in required_a3ms])

{'5', '7'}

In [90]:
len(required_a3ms)

1259

In [ ]:
# go and fold the proteins with the missing replicates 

In [14]:
metrics = pickle.load(open('/home/grig0076/scratch/phlegm/PHROGs/phrog_representative_structures/nonsingleton_representative_sequences.part_006.phlegm/need_updating/metrics_pickle/phrog_32002.pkl', 'rb'))

In [77]:
required_a3ms

['phrog_10901.4.a3m',
 'phrog_10905.4.a3m',
 'phrog_10906.4.a3m',
 'phrog_10908.4.a3m',
 'phrog_10909.4.a3m',
 'phrog_1090.4.a3m',
 'phrog_10914.4.a3m',
 'phrog_10916.4.a3m',
 'phrog_10918.4.a3m',
 'phrog_1091.4.a3m',
 'phrog_10921.4.a3m',
 'phrog_10926.4.a3m',
 'phrog_10928.4.a3m',
 'phrog_1092.4.a3m',
 'phrog_10934.4.a3m',
 'phrog_10936.4.a3m',
 'phrog_10937.4.a3m',
 'phrog_1093.4.a3m',
 'phrog_10941.4.a3m',
 'phrog_10944.4.a3m',
 'phrog_10945.4.a3m',
 'phrog_10946.4.a3m',
 'phrog_1094.4.a3m',
 'phrog_10950.4.a3m',
 'phrog_10951.4.a3m',
 'phrog_10952.4.a3m',
 'phrog_10955.4.a3m',
 'phrog_10957.4.a3m',
 'phrog_10958.4.a3m',
 'phrog_10959.4.a3m',
 'phrog_1095.4.a3m',
 'phrog_10960.4.a3m',
 'phrog_10966.4.a3m',
 'phrog_10967.4.a3m',
 'phrog_10971.4.a3m',
 'phrog_10976.4.a3m',
 'phrog_10977.4.a3m',
 'phrog_10978.4.a3m',
 'phrog_10979.4.a3m',
 'phrog_1097.4.a3m',
 'phrog_10980.4.a3m',
 'phrog_10981.4.a3m',
 'phrog_10982.4.a3m',
 'phrog_10983.4.a3m',
 'phrog_10984.4.a3m',
 'phrog_10985.4.a

In [36]:
set(replicate_strings) - set(replicates_present) 

set()